# Dados para testes no desenvolvimento

Este notebook gera dados para testar etapas do desenvolvimento de um pacote que treina e preve com o modelo. Os seguintes dados foram gerados:

- **Precisão do modelo para as previsões** : obtida pela previsão do modelo com os dados do arquivo "dados_teste.csv".
- **Tabela com dados de previsões** : esta tabela consiste dos dados no arquivo "dados_teste.csv" acrescido de duas coluna: 


In [2]:
import pandas as pd
import numpy as np
import joblib
from sklearn import metrics

In [3]:
# Classes para transformação dos dados
# =====================================


from sklearn.base import BaseEstimator, TransformerMixin

class Mapeamento(BaseEstimator, TransformerMixin):    

    def __init__(self, variavel, mapa):        
        '''
        variavel: nome da coluna que será alterada
        mapa: mapa com as alterações
        '''
        self.variavel = variavel
        self.mapa = mapa 

        
    def fit(self, X, y=None):        
        '''
        X: dataframe com os dados de treinamento        
        y: serie com o rótulos do dados de treinamento
        '''        
        return self
    

    def transform(self, X):
        '''
        X: dataframe com os dados de treinamento        
        '''        
        X = X.copy()        
        X[self.variavel] = X[self.variavel].map(self.mapa)
        return X
    
    
    
    
    
class transf_yeo_johnson(BaseEstimator, TransformerMixin):    

    def __init__(self, colunas):
        '''
        colunas: lista com as colunas que serão transformadas
        '''
        self.colunas = colunas
        self.obj_transf= PowerTransformer(method="yeo-johnson")
    
    
    def fit(self, X, y=None):
        '''
        X: dataframe com os dados de treinamento        
        y: serie com o rótulos do dados de treinamento
        '''        
        self.obj_transf= self.obj_transf.fit(X[self.colunas].values) 
        return self  
    

    def transform(self, X):
        '''
        X: dataframe com os dados de treinamento        
        '''
        X = X.copy()
        X_array = X[self.colunas].values
        X[self.colunas] = self.obj_transf.transform(X_array)
        return X

In [14]:
# carrega o modelo
# ==================

model = joblib.load('.//modelo_treinado//model_pipe.joblib') 

In [15]:
# importa os dados
# ====================

df = pd.read_csv(".//dados//dados_originais//dados_teste.csv" ,sep=',');
df.head(3)

,age,gender,height_cm,weight_kg,body fat_%,diastolic,systolic,gripForce,sit and bend forward_cm,sit-ups counts,broad jump_cm,class
0,26.0,M,177.5,77.84,19.5,66.0,130.0,49.2,20.3,60.0,242.0,A
1,35.0,M,177.8,77.00,19.0,76.0,126.0,43.0,10.1,45.0,246.0,B
2,25.0,M,174.0,77.60,24.4,77.0,157.0,48.6,18.6,59.0,211.0,A


In [16]:
# seleciona as variáveis
# ==============================

var_prev=[var for var in df.columns if var != 'class'] 

X = df[var_prev]     #variáveis para previsão
y = df['class']      #variável also

## Métrica: precisão do modelo

In [17]:
# modelos previstos
y_hat = model.predict(X)

print('Valores previstos: ', y_hat[0:10])
print('Valores reais: ', y.iloc[0:10].to_list())
print("Acuracia:", metrics.accuracy_score(y, y_hat))

Valores previstos:  ['A' 'B' 'B' 'B' 'B' 'D' 'B' 'D' 'D' 'A']
Valores reais:  ['A', 'B', 'A', 'B', 'B', 'D', 'C', 'D', 'D', 'B']
Acuracia: 0.7350746268656716


## Gera e salva a nova tabela

In [19]:
# previsão
y_hat=model.predict(X)

#acrescenta na tabela
df['prev_val']=y_hat

#verificação
df.head()['prev_val']

0    A
1    B
2    B
3    B
4    B
Name: prev_val, dtype: object

In [20]:
# salva a nova tabela
df.to_csv(".//dados//dados_test_pacote//dados_test_pacote.csv")